In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from tqdm import tqdm_notebook


import time
import pandas as pd
import numpy as np


## 1. '입찰정보'로 화면전환

In [3]:
path = '../chromedriver/chromedriver'
driver = webdriver.Chrome(path)
driver.get('http://www.g2b.go.kr')

time.sleep(5)

# 셀레니움은 iframe에 직접 접근하지 못함
# 그래서 frame으로 변환해 주어야 접근할 수 있음
# iframe의 id값을 이용해 frame으로 변환

driver.switch_to.frame('maintop_iframe')

# '입찰정보'로 화면전환
iframe = driver.find_element_by_xpath('//*[@id="gnb_logout"]/ul/li[1]/a/img')
iframe.click()

time.sleep(3)

## 2. '공고명', '참가제한지역' 입력
   - 공고명에는 '통신', 참가제한지역에는 '서울'을 입력

In [4]:
# 이것 또한 frame태그에 접근하기 위한 코드
driver.switch_to.frame('sub')
driver.switch_to.frame('main')

gongo_myung = driver.find_element_by_xpath('//*[@id="bidNm"]')
gongo_myung.send_keys('통신')

arae = Select(driver.find_element_by_xpath('//*[@id="area"]'))
arae.select_by_value('11')

time.sleep(2)

driver.find_element_by_xpath('//*[@id="buttonwrap"]/div/a[1]').click()

## 3. 게시물 클릭하고 데이터 크롤링하기

In [5]:
post_data = []
now = time.strftime('%Y/%m/%d %H:%M', time.localtime(time.time()))
while 1 :
    try:
        driver.switch_to.frame('sub')
        driver.switch_to.frame('main')
    except:
        pass
    
    gongo = driver.find_element_by_xpath('//*[@id="resultForm"]/div[2]/table/tbody')
    gongo_list = np.array(gongo.find_elements_by_css_selector('tr'))
    
    daed_line_temp = [temp.find_element_by_css_selector('td.tc > div > span').text for temp in gongo_list]
    dead_line = np.array(list(map(lambda x: x[1:len(x)-1], daed_line_temp)))
    index = dead_line > now
    ing_list = np.array(range(len(gongo_list)))[index]
    
    if len(ing_list)==0:
        break
    
    for idx in tqdm_notebook(ing_list):

        gongo_list[idx].find_element_by_css_selector('td.tl > div > a').click()

        # 1. 공고일반
        table_01 = driver.find_element_by_xpath('//*[@id="container"]/div[5]/table/tbody')

        tr_list = table_01.find_elements_by_css_selector('tr')
        data_dict = {}

        time.sleep(1)

        for tr in tr_list:
            th_list = tr.find_elements_by_css_selector('th')
            td_list = tr.find_elements_by_css_selector('td')

            for i in range(len(th_list)):
                key = th_list[i].text
                value = td_list[i].text
                data_dict[key] = value

        time.sleep(1)

        # 2. 입찰집행 및 진행 정보
        table_02 = driver.find_element_by_xpath('//*[@id="container"]/div[7]/table/tbody')

        tr_list = table_02.find_elements_by_css_selector('tr')

        for tr in tr_list:
            th_list = tr.find_elements_by_css_selector('th')
            td_list = tr.find_elements_by_css_selector('td')

            for i in range(len(th_list)):
                key = th_list[i].text
                value = td_list[i].text
                data_dict[key] = value    

        time.sleep(1)

        # 3. 예정가격 결정 및 입찰금액 정보
        table_03 = driver.find_element_by_xpath('//*[@id="container"]/div[9]/table/tbody')

        tr_list = table_02.find_elements_by_css_selector('tr')

        for tr in tr_list:
            th_list = tr.find_elements_by_css_selector('th')
            td_list = tr.find_elements_by_css_selector('td')

            for i in range(len(th_list)):
                key = th_list[i].text
                value = td_list[i].text
                data_dict[key] = value    

        time.sleep(1)

        # 4. 기초금액공개
        try:
            basic_price = driver.find_element_by_css_selector('table.table_list_baseEstiPriceTbl > tbody > tr > td.tr').text
        except :
            basic_price = ''
        
        data_dict['기초금액'] = basic_price
        
        
        post_data.append(data_dict)

        driver.back()
        time.sleep(1)

        # 뒤로가기 하면 'gongo', 'gongo_list'초기화됨
        driver.switch_to.frame('sub')
        driver.switch_to.frame('main')
        gongo = driver.find_element_by_xpath('//*[@id="resultForm"]/div[2]/table/tbody')
        gongo_list = gongo.find_elements_by_css_selector('tr')        

        
    try :
        driver.find_element_by_xpath('//*[@id="pagination"]/a').click()
    except:
        break
    
    time.sleep(2)

In [6]:
post_data

[{'공고종류': '실공고',
  '게시일시': '2020/12/24 20:52',
  '입찰공고번호': '20201238499 - 00',
  '참조번호': '항대 20-44호 (C)',
  '공고명': '전자관 4층 환경개선 정보통신공사\n본 공고는 지문인식 전자입찰제도가 적용되오니 미리 지문보안토큰에 지문정보를 등록하여야 합니다.\n본 공고는 "나라장터 안전 입찰서비스"를 이용하여야만 전자입찰서를 제출 할 수 있습니다. 유의사항 안내',
  '공고기관': '한국항공대학교',
  '수요기관': '한국항공대학교',
  '입찰방식': '전자입찰',
  '낙찰방법': '공고서 참조',
  '계약방법': '제한경쟁',
  '국제입찰구분': '국내입찰',
  '재입찰': '재입찰 허용',
  '발주계획번호': '3-1-2020-7002246-000013',
  '공사현장': '경기도 고양시 덕양구',
  '국내/국제 입찰사유': '국제입찰 비대상(고시금액 이하 또는 대상기관아님)',
  'WTO품목번호': '건설공사',
  '입찰자격': '공고서 참조',
  '관련공고': '',
  '공고담당자': '손동기 (02-300-0213)',
  '집행관': '손동기',
  '입찰개시일시': '2020/12/24 21:00:00',
  '입찰마감일시': '2021/01/04 11:00:00',
  '개찰(입찰)일시': '2021/01/04 13:00:00',
  '개찰장소': '국가종합전자조달시스템(나라장터)',
  '입찰참가자격등록\n마감일시': '2020/12/31 18:00:00\n조달청 입찰참가등록 가능시간은 평일 09:00~18:00 이며, 토요일, 일요일 및 공휴일은 업무처리가 불가합니다.\n본 입찰에 참여하는 업체는 입찰참가자격등록마감일시까지 나라장터에 경쟁입찰참가자격등록(특히 물품, 업종 등 자격제한 사항 필히 확인)을 해야 합니다.',
  '보증서접수마감일시': '보증서 접수마감일시를 입력하지 않은 경우에는, 입찰서 접수마감일 전일 18시까지 제출이 가능합

In [7]:
# 공고번호, 공고명, 기관 ,입찰개시일시, 입찰마감일시, 기초가격


data = {
    '공고번호'     : [temp['입찰공고번호'] for temp in post_data],
    '공고명'       : [temp['공고명'] for temp in post_data],
    '공고기관'     : [temp['공고기관'] for temp in post_data],
    '입찰개시일시' : [temp['입찰개시일시'] for temp in post_data],
    '입찰마김일시' : [temp['입찰마감일시'] for temp in post_data],
    '기초금액'     : [temp['기초금액'] for temp in post_data]
}

df = pd.DataFrame(data)

In [8]:
df['공고명'] = df['공고명'].str.split('\n').map(lambda x : x[0])

In [9]:
df

,공고번호,공고명,공고기관,입찰개시일시,입찰마김일시,기초금액
0,20201238499 - 00,전자관 4층 환경개선 정보통신공사,한국항공대학교,2020/12/24 21:00:00,2021/01/04 11:00:00,


In [10]:
df.to_csv('나라장터크롤링[2020-12-29].csv', encoding='UTF-8-SIG')